In [1]:
import pandas as pd
import requests
from common_func import *
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore"
list_class = "mw-body-content mw-content-ltr"

In [3]:
res = requests.get(url)

In [4]:
res_text = res.text

In [5]:
soup = BeautifulSoup(res_text, 'html.parser')
data = [x.find_all('a') for x in soup.find_all(class_="div-col")]

In [6]:
malls = [x.get('title') for y in data for x in y]

In [7]:
malls = [x.lower() for x in malls if x is not None]

In [8]:
mall_df = pd.DataFrame(malls,columns=['Mall'])

In [9]:
remove_words = ['shopping mall','singapore']
mall_df['Mall'] = mall_df['Mall'].str.replace("|".join(remove_words),'',regex = True)

In [10]:
mall_df

,Mall
0,bugis junction
1,bugis+
2,capitol piazza
3,cathay cineleisure orchard
4,the centrepoint
...,...
82,pioneer mall
83,one-north
84,gek poh shopping centre
85,west coast plaza


In [11]:
mall_df['Coords']= mall_df['Mall'].apply(getcoordinates)

In [12]:
to_replace = mall_df[mall_df['Coords'].isna()]['Mall'].values

In [13]:
to_replace

array(['shaw house and centre', 'the adelphi,  (page does not exist)',
       'ntuc downtown east'], dtype=object)

In [14]:
mall_df= mall_df.replace(list(to_replace), ['shaw house',
                   'the adelphi',
                   'downtown east'])


In [15]:
for count, row in mall_df.iterrows():

    if row['Coords'] is None:
        row['Coords'] = getcoordinates(row['Mall'])

In [16]:
create_lat_lon(mall_df).to_csv('amenities/malls.csv')

# Mrt

In [17]:
url2 = "https://mrtmapsingapore.com/mrt-stations-singapore/"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.79 Safari/537.36'}

In [18]:
res2 = requests.get(url2,headers = headers).text

In [19]:
soup2 = BeautifulSoup(res2,'html.parser')
mrt_list = soup2.find(class_ = "tablepress").find_all('td','column-2')

In [20]:
new_mrt_list = [x.string.strip().lower() + ' mrt' for x in mrt_list]

In [21]:
mrt_df = pd.DataFrame(new_mrt_list,columns = ['Station'])

In [22]:
mrt_df['Coords'] = mrt_df['Station'].apply(getcoordinates)

In [23]:
mrt_df[mrt_df['Coords'].isna()]

,Station,Coords


In [25]:
create_lat_lon(mrt_df).to_csv('amenities/mrt.csv')